In [204]:
import trimesh
from trimesh import viewer
from trimesh.viewer import windowed
from trimesh import creation, transformations
import numpy as np
import math
import sys
from scipy.spatial.transform import Rotation as R
import random

In [230]:
#Set up all parameters to be used in the planner here
pi = math.pi
yaw_rate_bounds = [-pi/6, pi/6]
pitch_rate_bounds = [-pi/6, pi/6]
lin_acc_rate_bounds = [-0.5, 0.5]

pitch_bounds = [-pi/3,pi/3]
velocity_bounds = [-1,1]
delta_s = 1.2
delta_bn = 3
T_prop = 2
probability_goal = 0.05
dynamic_bounds = {"PITCH":pitch_bounds,"VEL":velocity_bounds}
goal_state = [15, -300, 5, 0,0,0]
goal_radius = 2

N = 10000 #number of iterations
STATE_SPACE = {"X":None,"Y":None,"Z":None,"PITCH":None,"YAW":None,"VEL":None}
U = {"PITCH_RATE":yaw_rate_bounds,"YAW_RATE":pitch_rate_bounds,"LIN_ACC":lin_acc_rate_bounds}

In [206]:
#### LOAD IN STL MODELS ####
#########print("Loading environment stl...")
envMesh = trimesh.load_mesh("C:/Users/mcgra/source/repos/AMP_FinalProject_Python/AMP_FinalProject_Python/FlatSlopes_WithTrees.stl",process=False)
envMesh = trimesh.Trimesh(envMesh.vertices,envMesh.faces,envMesh.face_normals,envMesh.vertex_normals)
#####print("Done.")


In [207]:
print("Loading drone stl...")
droneSTL = trimesh.load_mesh("C:/Users/mcgra/source/repos/AMP_FinalProject_Python/AMP_FinalProject_Python/DroneGeneral_Smaller.stl")
droneSTL = trimesh.Trimesh(droneSTL.vertices,droneSTL.faces,droneSTL.face_normals,droneSTL.vertex_normals)
print("Done.")
#Get max dimension of the drone
bounds = droneSTL.bounds
x_dim = bounds[1][0]-bounds[0][0]
y_dim = bounds[1][1]-bounds[0][1]
z_dim = bounds[1][2]-bounds[0][2]
droneMaxDim = math.sqrt(x_dim**2+y_dim**2+z_dim**2)
print("Drone Max Dimension: " + str(droneMaxDim))

Loading drone stl...
Done.
Drone Max Dimension: 0.4856526011604424


In [208]:

print("Loading bounding mesh stl...")
boundingMesh = trimesh.load_mesh("C:/Users/mcgra/source/repos/AMP_FinalProject_Python/AMP_FinalProject_Python/FlatSlopes_BoundingVolume.stl")
boundingMesh = trimesh.Trimesh(boundingMesh.vertices,boundingMesh.faces,boundingMesh.face_normals,boundingMesh.vertex_normals)
print("Done.")


Loading bounding mesh stl...
Done.


In [209]:

print("Loading drone obj...")
droneMesh = trimesh.load_mesh("C:/Users/mcgra/source/repos/AMP_FinalProject_Python/AMP_FinalProject_Python/DroneGeneral_Smaller.obj")
#droneMesh = trimesh.Trimesh(droneMesh.vertices,droneMesh.faces,droneMesh.face_normals,droneMesh.vertex_normals)
print("Done.")


Loading drone obj...
Done.


In [207]:
print("Loading drone stl...")
droneSTL = trimesh.load_mesh("C:/Users/mcgra/source/repos/AMP_FinalProject_Python/AMP_FinalProject_Python/DroneGeneral_Smaller.stl")
droneSTL = trimesh.Trimesh(droneSTL.vertices,droneSTL.faces,droneSTL.face_normals,droneSTL.vertex_normals)
print("Done.")
#Get max dimension of the drone
bounds = droneSTL.bounds
x_dim = bounds[1][0]-bounds[0][0]
y_dim = bounds[1][1]-bounds[0][1]
z_dim = bounds[1][2]-bounds[0][2]
droneMaxDim = math.sqrt(x_dim**2+y_dim**2+z_dim**2)
print("Drone Max Dimension: " + str(droneMaxDim))

Loading drone stl...
Done.
Drone Max Dimension: 0.4856526011604424


In [210]:
def Runge_Kutta_Integration_fourth_order(T_sampled,input_array,dynamic_bounds):
    None

def Get3Ddist(xyz1,xyz2):
    dist = math.sqrt((xyz1[0]-xyz2[0])**2 + (xyz1[1]-xyz2[1])**2 + (xyz1[1]-xyz2[1])**2)
    # print("DIST: " + str(dist))
    return dist


def CreateRotationMatrix(alpha = 0, beta = 0, gamma = 0):
    #NO INPUTS CREATES IDENTITY MATRIX
    r_x = transformations.rotation_matrix(alpha,[1,0,0])
    r_y = transformations.rotation_matrix(beta,[0,1,0])
    r_z = transformations.rotation_matrix(gamma,[0,0,1])
    R = np.matmul(np.matmul(r_z,r_y),r_x)
    return R


def CombineRotTransMatrices(rotationMatrix,translationMatrix):
    rotMatShape = np.shape(rotationMatrix)
    tranMatShape = np.shape(translationMatrix)
    if(rotMatShape != (4,4) and rotMatShape != (3,3)):
        raise Exception("Expecting rotation matrix with shape (4, 4) or (3, 3), given matrix of shape "+str(rotMatShape))
        sys.exit()
    elif(tranMatShape != (4,4) and tranMatShape != (4,1)):
        raise Exception("Expecting translation Matrix matrix with shape (4, 4) or (4, 1), given matrix of shape "+str(tranMatShape))
        sys.exit()

        
    tranMat = translationMatrix[0:3,3]
    combinedMatrix = np.zeros([4,4])

    if(rotMatShape != (3,3)):
        rotMat = rotationMatrix[0:3,0:3]
        rotMatTrans = rotationMatrix[0:3,3]
        combinedMatrix[0:3,3] = tranMat + rotMatTrans
    else:
        rotMat = rotationMatrix
        combinedMatrix[0:3,3] = tranMat

    combinedMatrix[0:3,0:3] = rotMat
    combinedMatrix[3,:] = np.array([0,0,0,1])
    return combinedMatrix

def CheckForDroneCollision(collision_manager_with_env = trimesh.collision.CollisionManager().__init__,drone_mesh=None,transform=None):
    collision_manager_with_env.add_object("Drone",drone_mesh,transform=transform)
    isCollision = collision_manager_with_env.in_collision_internal()
    collision_manager_with_env.remove_object("Drone")
    return isCollision

def CalculateFinalTransform(TransformationList):
    #Transformation list is a list of transformations from 1st to last
    #THIS WONT WORK -  NEED TO FIGURE OUT

    prevTransform = np.zeros([4,4])
    for i in range(len(TransformationList),0,-1):
        if(i == 0):
            break
        else:
            if(i == len(TransformationList)):
                prevTransform = np.matmul(TransformationList[i],TransformationList[i-1])
            else:
                prevTransform = np.matmul(prevTransform,TransformationList[i-1])

    return prevTransform ####XXXXXX

In [211]:
class MeshObject:
    def __init__(self,mesh,name):
        self.name = name
        self.mesh = mesh

        self.scene = trimesh.Scene()
        self.scene.add_geometry(mesh)
        self.centroid = self.scene.centroid
        self.X = self.centroid[0]
        self.Y = self.centroid[1]
        self.Z = self.centroid[2]
        self.alpha = 0 #rotation about X, roll
        self.beta = 0  #rotation about Y, pitch
        self.gamma = 0 #rotation about Z, yaw
        self.velocity = 0
        self.ROTATION_MATRIX = np.array([[1,0,0],[0,1,0],[0,0,1]])
        self.TRANSLATION_MATRIX = np.array([self.X,self.Y,self.Z])
    
    def GetEulerAnglesFromMatrix(self):
        r = R.from_matrix(self.ROTATION_MATRIX)
        xyz = r.as_euler('xyz',degrees=False)
        self.alpha = xyz[0]
        self.beta = xyz[1]
        self.gamma = xyz[2]

    def RotateMesh(self,rot_mat):
        self.ROTATION_MATRIX = np.matmul(self.ROTATION_MATRIX,rot_mat)
        self.GetEulerAnglesFromMatrix()

    def TranslateMesh(self,transl_mat):
        self.TRANSLATION_MATRIX = np.add(self.TRANSLATION_MATRIX,transl_mat)
        self.X = self.TRANSLATION_MATRIX[0]
        self.Y = self.TRANSLATION_MATRIX[1]
        self.Z = self.TRANSLATION_MATRIX[2]
        
    def GetMeshCentroid(self,mesh):
        scene = trimesh.Scene()
        scene.add_geometry(self.mesh)
        centroid = scene.centroid
        return centroid

    def CalculateTrajectory():
        None

In [212]:
class Vertice:
    def __init__(self,state,cost):
        self.state=state
        self.cost = cost
        self.Parent = None
        self.Children = []

    def AddParent(self,parent):
        self.Parent = parent
    def AddChild(self,child):
        self.Children.append(child)


In [213]:
class PlanningAlgorithms:
    def __init__(self):
        None

    class SST: #We will be using the drone model from the exam
        def __init__(self,start_state=None,goal_state=None,u_bounds_list=None,StateSpace=None):
            self.start_state = start_state
            self.goal_state = goal_state
            self.Witness_Regions = self.WitnessRegions()
            self.Active_Set = self.ActiveSet()
            self.Inactive_Set = self.InactiveSet()
            #self.Active_Set.Has_Witness = False
            self.StateSpace = StateSpace
            None
        def SST(X,U,x_0,T_prop,N,delta_BestNeighbor,delta_s):
            None

        def Is_Node_Locally_Best_SST(X_new,S,delta_s):
            None
        def Prune_Dominated_Nodes_SST(X_new,V_active,V_inactive,E):
            None
        def Best_First_Selection_SST(X,V,delta_BestNeighbor):
            x_rand = StateValidation.StateSample(boundingMesh,pitch_bounds=pitch_bounds,velocity_bounds=velocity_bounds)
            x_nearList = [(Get3Ddist(V[i][0:3],x_rand[0:3]),i) for i in range(0,len(V)) if Get3Ddist(V[i][0:3],x_rand[0:3]) < delta_BestNeighbor]
            x_near = [V[i] for i in range(0,len(V)) if Get3Ddist(V[i][0:3],x_rand[0:3]) < delta_BestNeighbor]
            if(len(x_near) == 0): #no neighbors within delta_BestNeighbor
                None

        def MonteCarlo_Prop(x_selected,U,T_prop):
            #x_selected is chosen neighbor, U is input bounds dict, T is max propagation time
            numInputs = len(U)
            random_inputs = np.empty([1,numInputs])
            for input in range(0,numInputs):
                inputBounds = input.value #get bounds of input from dictionary
                inputSample = random.uniform(inputBounds[0],inputBounds[1])
                random_inputs[0][input] = random.uniform(U[input][0],U[input][1])
            t = random.uniform(0,T_prop)

        def Randomized_Inputs(u_bounds_list):
            random_inputs = np.empty([1,len(u_bounds_list)])
            numInputs = len(u_bounds_list)
            
            for i in range(0,numInputs):
                random_inputs[0][i] = random.uniform(u_bounds_list[i][0],u_bounds_list[i][1])
            return random_inputs

        class ActiveSet:
            def __init__(self):
                self.States = []
                self.States.append(self.start_state)
                None
        class InactiveSet:
            def __init__(self):
                None
        class WitnessRegions:
            def __init__(self,x_0,d_s,cost):
                self.Regions = []
                region = self.Region(x_0,d_s,cost)
                self.Regions.append(region)

            def AddNewRegion(self,rep,d_s,cost):
                region = self.Region(rep,d_s,cost)
                self.Regions.append(region)

            class Region:
                def __init__(self,rep,d_s,cost):
                    self.representative = rep #current state that represents the region
                    x = rep.state[0]
                    y = rep.state[1]
                    z = rep.state[2]
                    self.cost = cost
                    regionSphere = trimesh.creation.icosphere(3,d_s)
                    regionSphere.apply_translation([x,y,z])
                    self.regionMesh = regionSphere
                    
                def ChangeRepresentative(self,x,cost):
                    self.representative = x
                    self.cost = cost

In [214]:
droneMeshOBJ = MeshObject(droneMesh,"DRONE")
x_0 = [droneMeshOBJ.X,droneMeshOBJ.Y,droneMeshOBJ.Z,droneMeshOBJ.beta,droneMeshOBJ.gamma,droneMeshOBJ.velocity]
envMeshOBJ = MeshObject(envMesh,"ENV")
boundaryMeshOBJ = MeshObject(boundingMesh,"BOUNDARY")


In [212]:
class Vertice:
    def __init__(self,state,cost):
        self.state=state
        self.cost = cost
        self.Parent = None
        self.Children = []

    def AddParent(self,parent):
        self.Parent = parent
    def AddChild(self,child):
        self.Children.append(child)


In [231]:
E = [] #trajectory list
V_active = [] #active state list
initialState = Vertice(x_0,0)
V_active.append(initialState)
V_inactive = [] #inactive state list
V = V_active+V_inactive #active and inactive states
G = [V,E]
S = PlanningAlgorithms.SST.WitnessRegions(initialState,delta_s,0)
print(V[0].state[0:3])

[300.0, -15.000001000000001, 5.0001225]


In [223]:
#CREATE THE COLLISION MANAGER WITH OUR ENVIRONMENT
print("Creating collision manager with environment...")
ENV_COLIS_MNGR = trimesh.collision.CollisionManager()
ENV_COLIS_MNGR.add_object("ENV",envMesh)
print("Done.")

Creating collision manager with environment...
Done.


In [232]:
class StateSampler:
    def __init__(self,boundingMesh):
        self.boundingMesh = boundingMesh

    def StateSample(self,x_bounds=None,y_bounds=None,z_bounds=None,pitch_bounds=None,beta_bounds=None,yaw_bounds=None,velocity_bounds=None):
            if(self.boundingMesh is not None):
                boundingMesh = self.boundingMesh
                sample = None
                while(sample is None or len(sample) == 0):
                    sample = trimesh.sample.volume_mesh(boundingMesh,1)
                x = sample[0][0]
                y = sample[0][1]
                z = sample[0][2]
                #alpha = random.uniform(alpha_bounds[0],alpha_bounds[1])
                pitch = random.uniform(pitch_bounds[0],pitch_bounds[1])
                yaw = random.uniform(0,2*pi)
                velocity = random.uniform(velocity_bounds[0],velocity_bounds[1])

            else:
                x = random.uniform(x_bounds[0], x_bounds[1])
                y = random.uniform(y_bounds[0], y_bounds[1])
                z = random.uniform(z_bounds[0], z_bounds[1])
                #alpha = random.uniform(alpha_bounds[0],alpha_bounds[1])
                pitch = random.uniform(pitch_bounds[0],pitch_bounds[1])
                yaw = random.uniform(0,2*pi)
                velocity = random.uniform(velocity_bounds[0],velocity_bounds[1])

            return [x,y,z,pitch,yaw,velocity]

In [233]:
def ExtendBranch(state0,state1,delta_bn):
    #Get unit vector <x,y,z>
    state_diff = np.subtract(state1,state0)
    print("STATE 0: " +str(state0))
    # print(state_diff)
    vector = state_diff[0:3]
    abs_vector = math.sqrt(vector[0]**2+vector[1]**2+vector[2]**2)
    unit_vector = vector/abs_vector
    extension = unit_vector*delta_bn
    
    new_xyz = state0[0:3]+extension
    x_new = [new_xyz[0],new_xyz[1],new_xyz[2],state1[3],state1[4],state1[5]]
    return x_new

In [234]:
def Best_First_Selection_SST(X,V,delta_BestNeighbor,sampler):
            prob = random.uniform(0,1) 
            #need to include probability of x_rand being the goal state
            #to occasionaly attempt to reach the goal
            if(prob <= probability_goal):
                x_rand = goal_state
            else:
                x_rand = sampler.StateSample(sampler.boundingMesh,pitch_bounds=pitch_bounds,velocity_bounds=velocity_bounds)
            # print(x_rand)
            # x_nearList = [(Get3Ddist(V[i].state[0:3],x_rand[0:3]),i) for i in range(0,len(V)) if Get3Ddist(V[i].state[0:3],x_rand[0:3]) < delta_BestNeighbor]
            x_nearList = [[V[i].cost,i] for i in range(0,len(V)) if Get3Ddist(V[i].state[0:3],x_rand[0:3]) < delta_BestNeighbor]
            x_near = [[int(i),Get3Ddist(V[i].state[0:3],x_rand[0:3])] for i in range(0,len(V))]
            x_near = sorted(x_near, key = lambda x: x[1])
            # print("X_NEARLIST: "+str(x_near))
            closest = int(x_near[0][0])
            # print("CLOSEST NEIGHBOR: " x)
            # print(closest)
            if(len(x_nearList) == 0): #no neighbors within delta_BestNeighbor, use closest neighbor and extend branch by delta_BN
                x_new = ExtendBranch(V[closest].state,x_rand,delta_BestNeighbor)
                # print("X_NEW: "+str(x_new))
                mode = "Branch Extension"
                return x_new, closest, mode
            else:
                x_nearList.sort(key=lambda x: x[0])
                # print("X NEAR LIST: " +str(x_nearList))
                x_new_parent_idx = x_nearList[0][1]
                mode = "Best Neighbor"
                return x_rand, x_new_parent_idx, mode

In [235]:
def Is_Node_Locally_the_Best_SST(xnew, S = PlanningAlgorithms.SST.WitnessRegions.__init__, delta_s=1):
    rgnIdx = 0
    for witness_region in S.Regions: #check if new or old cost is better and make that one region rep
        if(witness_region.regionMesh.contains(([xnew.state[0:3]]))):
            if(witness_region.cost > xnew.cost):
                witness_region.ChangeRepresentative(xnew,xnew.cost)
                inOldRegion = True
                return True, rgnIdx
            else:
                return False, rgnIdx
        rgnIdx += 1
            

    #if it reaches this point, it was not in an old region, new witness region will be created
    S.AddNewRegion(xnew,delta_s,xnew.cost)
    rgnIdx += 1
    return True, rgnIdx
        

In [236]:
def In_Goal_Region(state,goal_state=goal_state,goal_radius=goal_radius):
    Goal_Sphere = trimesh.creation.icosphere(radius=goal_radius)
    Goal_Sphere = Goal_Sphere.apply_translation(goal_state[0:3])
    if(Goal_Sphere.contains([state[0:3]])):
        return True
    else:
        return False

In [237]:
# x,y,z = trimesh.sample.volume_mesh(boundingMesh,1)[0]
# print(x)
# print(y)
# print(z)

ss_sampler = StateSampler(boundingMesh)
x_selected, x_new_parentIdx, mode = Best_First_Selection_SST(STATE_SPACE,V_active,delta_bn,ss_sampler)
# print(x_selected)
# print(x_new_parentIdx)
# print(ss_sampler.StateSample(pitch_bounds=pitch_bounds,velocity_bounds=velocity_bounds))
scene = trimesh.Scene()
scene.add_geometry(envMesh)
for iteration in range(0,N):
    # print(iteration)
    x_selected, x_new_parentIdx, mode = Best_First_Selection_SST(STATE_SPACE,V_active,delta_bn,ss_sampler)
    print("PARENT INDEX: "+str(x_new_parentIdx)+", MODE: "+mode)
    x_selected_parent = V_active[int(x_new_parentIdx)].state
    x_new_cost = Get3Ddist(x_selected,x_selected_parent)+V_active[int(x_new_parentIdx)].cost
    x_new = Vertice(x_selected,x_new_cost)

    #CHECK IF SELECTED STATE IS COLLISION FREE
    #USING A SPHERE TO REPRESENT THE DRONE, USING THE DRONES MAXIMUM DIMENSION  IS THE SIMPLEST OPTION,
    #AS WE CAN CREATE THE SPHERE AT THE SPECIFIED XYZ TO HAVE IT REPRESENT ALL ORIENTATIONS
    #AND INSERT IT AND REMOVE IT FROM THE COLLISION MANAGER WITHOUT HAVING TO PERFORM TRANSFORMATIONS
    droneSphere = trimesh.creation.icosphere(3,droneMaxDim,(0,0,255))
    droneSphere = droneSphere.apply_translation(x_selected[0:3])
    # print(droneSphere.centroid)
    dronePosCollision = CheckForDroneCollision(ENV_COLIS_MNGR,droneSphere)

    #Need to check for collision in trajectory, can use cylinder to represent space the drone travels through
    trajCylinder = trimesh.creation.cylinder(droneMaxDim,segment=[x_selected[0:3],x_selected_parent[0:3]])
    trajCylinder.visual.face_colors = [255, 0, 0, 255]
    droneTrajCollision = CheckForDroneCollision(ENV_COLIS_MNGR,trajCylinder)
    # scene = trimesh.Scene()
    # scene.add_geometry(trajCylinder)
    # scene.add_geometry(droneSphere)
    # scene.add_geometry(envMesh)
    # scene.show()

    # if(dronePosCollision==True):
    #     print("POS COLLISION")
    # if(droneTrajCollision==True):
    #     print("TRAJ COLLISION")

    #IF NO COLLISION OCCURS, WE CAN MOVE ONTO CHECKING IF VERTICE IS BEST IN WITNESS REGION
    if(droneTrajCollision == False and dronePosCollision==False):
        isNodeBest, rgnIdx = Is_Node_Locally_the_Best_SST(x_new,S=S,delta_s=delta_s)
        if(isNodeBest):
            x_new.Parent = x_selected_parent
            V_active[int(x_new_parentIdx)].Children.append(x_new)
            V_active.append(x_new)

            E.append([V_active[int(x_new_parentIdx)],x_new])
            scene.add_geometry(droneSphere)
            scene.add_geometry(trajCylinder)
            # if(In_Goal_Region(x_new.state)):
            #     print("GOAL REACHED")
            #     break

        # print("NO COLLISION")
# for vertice in V_active:
#     print(vertice.state)

STATE 0: [300.0, -15.000001000000001, 5.0001225, 0, 0, 0]
STATE 0: [300.0, -15.000001000000001, 5.0001225, 0, 0, 0]
PARENT INDEX: 0, MODE: Branch Extension
STATE 0: [297.87867965271886, -17.12132133983793, 5.000121588204412, 0, 0, 0]
PARENT INDEX: 1, MODE: Branch Extension
STATE 0: [296.99125180192544, -19.983677642883656, 5.1393631195466964, 0.9912674102414829, 6.258786623311354, 0.1135556286786128]
PARENT INDEX: 2, MODE: Branch Extension
STATE 0: [296.2337348940475, -22.886462063147704, 5.136085353334889, 0.2438568607672329, 4.944129835146946, 0.03701341341614239]
PARENT INDEX: 3, MODE: Branch Extension
STATE 0: [293.5969338117398, -24.31560654009942, 5.205555608404539, -0.8467323147354445, 3.6435947894744163, -0.7854920671136396]
PARENT INDEX: 4, MODE: Branch Extension
STATE 0: [293.4980664494599, -27.312819832695126, 5.28884860548967, 0.37164649006948847, 1.66573082986455, -0.2466324051925226]
PARENT INDEX: 5, MODE: Branch Extension
STATE 0: [293.18526784188447, -30.296466542357614